In [ ]:
import tensorflow as tf

# Enable GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


import numpy as np
from matplotlib import pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Flatten, Dropout, Conv1D, MaxPooling1D, Bidirectional
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
drive.mount('/content/drive')


In [ ]:
#some preprocess functions:
def reshape_data(input_data):
  #take first 12 one's
    for i in range(0, len(input_data)):
      input_data[i] = input_data[i][:24]


    #remove the probability of each keypoint
    euclidean_data_list = []
    for video_data in input_data:
        euclidean_data = video_data[:, :, :2]
        euclidean_data_list.append(euclidean_data)

    input_data = euclidean_data_list
    indexes_to_delete = [1, 2, 3, 4, 13, 14, 15, 16]
    for i, video_data in enumerate(input_data):
        video_data = np.delete(video_data, indexes_to_delete, axis=1)
        input_data[i] = video_data

    #reshape to (batch numbers, 24,  34)
    max_frames = max(len(video) for video in input_data)
    reshaped_output = np.zeros((len(input_data), max_frames, 18))
    print(len(input_data))
    for i, video in enumerate(input_data):
        reshaped_video = video.reshape(video.shape[0], -1)
        reshaped_output[i, :len(video)] = reshaped_video
    normalized_data = (reshaped_output - np.mean(reshaped_output)) / np.std(reshaped_output)
    return reshaped_output

def preprocess_datasets(dataset_vars, split_amount):
    reshaped_data = []
    numerical_labels = []

    for var_names in dataset_vars:
        # Load dataset
        pose_keypoints_list = var_names[0]
        labels_list = var_names[1]

        # Reshape data
        reshaped_data.append(reshape_data(pose_keypoints_list))

        # Convert labels to numerical values
        numerical_labels.append(np.array([label_mapping[label] for label in labels_list]))

    # Concatenate all the data
    X = tf.convert_to_tensor(np.concatenate(reshaped_data, axis=0), dtype=tf.float32)
    y = tf.keras.utils.to_categorical(np.concatenate(numerical_labels))

    # Shuffle the data
    indices = np.random.permutation(len(X))
    indices_tensor = tf.convert_to_tensor(indices, dtype=tf.int32)
    X = tf.gather(X, indices_tensor)
    y = tf.gather(y, indices_tensor)

    # Split data into training and validation sets
    train_size = int(split_amount * len(X))
    val_size = len(X) - train_size

    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    return X_train, X_val, y_train, y_val


def plot_points_with_slider(points):
    # Create the initial scatter plot
    fig = go.Figure()

    # Add traces for each time step
    for t in range(24):
        fig.add_trace(go.Scatter(
            x=points[t, :, 0],
            y=points[t, :, 1],
            mode='markers',
            name=f'Time Step {t}',
            marker=dict(
                size=8,
                color='blue'
            )
        ))

    # Update layout settings
    fig.update_layout(
        title='Points Over Time',
        xaxis=dict(range=[0, 1]),
        yaxis=dict(range=[0, 1]),
        hovermode='closest',
        sliders=[{
            'currentvalue': {'prefix': 'Time Step: '},
            'steps': [{'label': str(t), 'method': 'update', 'args': [{'visible': [t == i for i in range(24)]}]} for t in range(24)]
        }]
    )

    # Show the plot
    fig.show()

In [ ]:
#load test data
pose_keypoints_test = np.load('/content/drive/MyDrive/ModelFiles/padding/pose_keypoints_list_thunder_nao2_pad.npy', allow_pickle=True)
labels_list_test = np.load('/content/drive/MyDrive/ModelFiles/padding/labels_list_thunder_nao2_pad.npy', allow_pickle=True)

for i, item in enumerate(labels_list_test):
    if item == 'corner-kick-red':
        labels_list_test[i] = 'Corner-kick Red'
    elif item == 'corner-kick-blue':
        labels_list_test[i] = 'Corner-kick Blue'
    elif item == 'goal-red':
        labels_list_test[i] = 'Goal Red'
    elif item == 'goal-blue':
        labels_list_test[i] = 'Goal Blue'
    elif item == 'goal-kick-red':
        labels_list_test[i] = 'Goal-kick Red'
    elif item == 'goal-kick-blue':
        labels_list_test[i] = 'Goal-kick Blue'
    elif item == 'kick-in-red':
        labels_list_test[i] = 'Kick-in Red'
    elif item == 'kick-in-blue':
        labels_list_test[i] = 'Kick-in Blue'
    elif item == 'player-exchange-red':
        labels_list_test[i] = 'Player exchange Red'
    elif item == 'player-exchange-blue':
        labels_list_test[i] = 'Player exchange Blue'
    elif item == 'pushing-free-kick-red':
        labels_list_test[i] = 'Pushing Free-kick Red'
    elif item == 'pushing-free-kick-blue':
        labels_list_test[i] = 'Pushing Free-kick Blue'
    elif item == 'full-time':
        labels_list_test[i] = 'Fulltime'

# define label_mapping once based on first dataset!!! important :)
label_mapping = {label: i for i, label in enumerate(set(labels_list_test))}

#reshape data
reshaped_data_test = reshape_data(pose_keypoints_test)
numerical_labels_test = np.array([label_mapping[label] for label in labels_list_test])

X_test = tf.convert_to_tensor(reshaped_data_test, dtype=tf.float32)
y_test = tf.keras.utils.to_categorical(numerical_labels_test)

#
indices = np.random.permutation(len(X_test))
indices_tensor = tf.convert_to_tensor(indices, dtype=tf.int32)
X_test = tf.gather(X_test, indices_tensor)
y_test = tf.gather(y_test, indices_tensor)

class_counts = tf.math.reduce_sum(y_test, axis=0)

# Print the number of datapoints per class
for i, count in enumerate(class_counts):
    print(f"Class {i}: {count}")

In [ ]:
#load training data:
labels_list_1 = np.load('/content/drive/MyDrive/ModelFiles/labels_list_thunder_new.npy', allow_pickle=True)
pose_keypoints_list_1 = np.load('/content/drive/MyDrive/ModelFiles/pose_keypoints_list_thunder_new.npy', allow_pickle=True)

for i, item in enumerate(labels_list_1):
    if item == 'corner-kick-red':
        labels_list_1[i] = 'Corner-kick Red'
    elif item == 'corner-kick-blue':
        labels_list_1[i] = 'Corner-kick Blue'
    elif item == 'goal-red':
        labels_list_1[i] = 'Goal Red'
    elif item == 'goal-blue':
        labels_list_1[i] = 'Goal Blue'
    elif item == 'goal-kick-red':
        labels_list_1[i] = 'Goal-kick Red'
    elif item == 'goal-kick-blue':
        labels_list_1[i] = 'Goal-kick Blue'
    elif item == 'kick-in-red':
        labels_list_1[i] = 'Kick-in Red'
    elif item == 'kick-in-blue':
        labels_list_1[i] = 'Kick-in Blue'
    elif item == 'player-exchange-red':
        labels_list_1[i] = 'Player exchange Red'
    elif item == 'player-exchange-blue':
        labels_list_1[i] = 'Player exchange Blue'
    elif item == 'pushing-free-kick-red':
        labels_list_1[i] = 'Pushing Free-kick Red'
    elif item == 'pushing-free-kick-blue':
        labels_list_1[i] = 'Pushing Free-kick Blue'
    elif item == 'full-time':
        labels_list_1[i] = 'Fulltime'

labels_list_2 = np.load('/content/drive/MyDrive/ModelFiles/labels_list_thunder_joey.npy', allow_pickle=True)
pose_keypoints_list_2 = np.load('/content/drive/MyDrive/ModelFiles/pose_keypoints_list_thunder_joey.npy', allow_pickle=True)
for i, item in enumerate(labels_list_2):
    if item == 'Goal_Blue':
        labels_list_2[i] = 'Goal Blue'
    elif item == 'Goal_Red':
      labels_list_2[i] = 'Goal Red'
    elif item == 'Kick_In_Blue':
      labels_list_2[i] = 'Kick-in Blue'
    elif item == 'Kick_In_Red':
      labels_list_2[i] = 'Kick-in Red'
    elif item == 'Pushing_Free_Kick_Blue':
      labels_list_2[i] = 'Pushing Free-kick Blue'
    elif item == 'Pushing_Free_Kick_Red':
      labels_list_2[i] = 'Pushing Free-kick Red'
    elif item == 'Goal_Kick_Red':
      labels_list_2[i] = 'Goal-kick Red'
    elif item == 'Goal_Kick_Blue':
      labels_list_2[i] = 'Goal-kick Blue'
    elif item == 'Corner_Kick_Red':
      labels_list_2[i] = 'Corner-kick Red'
    elif item == 'Corner_Kick_Blue':
      labels_list_2[i] = 'Corner-kick Blue'
    elif item == 'Player_Exchange_Red':
      labels_list_2[i] = 'Player exchange Red'
    elif item == 'Player_Exchange_Blue':
      labels_list_2[i] = 'Player exchange Blue'
    elif item == 'Full_Time_mcp':
      labels_list_2[i] = 'Fulltime'

# Create a combined array of labels and keypoints
combined_data = list(zip(labels_list_2, pose_keypoints_list_2))

# Shuffle the combined array
np.random.shuffle(combined_data)

# Extract the shuffled labels and keypoints
labels_list_2, pose_keypoints_list_2 = zip(*combined_data)

labels_list_2 = list(labels_list_2)
pose_keypoints_list_2 = list(pose_keypoints_list_2 )

# Get a certain number of data points
num_data_points = 250  # Replace with your desired number
labels_list_2 = labels_list_2[:num_data_points]
pose_keypoints_list_2 = pose_keypoints_list_2[:num_data_points]


#pose_keypoints_list_3 = np.load('/content/drive/MyDrive/ModelFiles/augmented_skeleton_keypoints_500.npy', allow_pickle=True)
#labels_list_3 = np.load('/content/drive/MyDrive/ModelFiles/augmented_labels_500.npy', allow_pickle=True)

pose_keypoints_list_nao = np.load('/content/drive/MyDrive/ModelFiles/pose_keypoints_list_thunder_test.npy', allow_pickle=True)
labels_list_nao = np.load('/content/drive/MyDrive/ModelFiles/labels_list_thunder_test.npy', allow_pickle=True)

for i, item in enumerate(labels_list_nao):
    if item == 'corner-kick-red':
        labels_list_nao[i] = 'Corner-kick Red'
    elif item == 'corner-kick-blue':
        labels_list_nao[i] = 'Corner-kick Blue'
    elif item == 'goal-red':
        labels_list_nao[i] = 'Goal Red'
    elif item == 'goal-blue':
        labels_list_nao[i] = 'Goal Blue'
    elif item == 'goal-kick-red':
        labels_list_nao[i] = 'Goal-kick Red'
    elif item == 'goal-kick-blue':
        labels_list_nao[i] = 'Goal-kick Blue'
    elif item == 'kick-in-red':
        labels_list_nao[i] = 'Kick-in Red'
    elif item == 'kick-in-blue':
        labels_list_nao[i] = 'Kick-in Blue'
    elif item == 'player-exchange-red':
        labels_list_nao[i] = 'Player exchange Red'
    elif item == 'player-exchange-blue':
        labels_list_nao[i] = 'Player exchange Blue'
    elif item == 'pushing-free-kick-red':
        labels_list_nao[i] = 'Pushing Free-kick Red'
    elif item == 'pushing-free-kick-blue':
        labels_list_nao[i] = 'Pushing Free-kick Blue'
    elif item == 'full-time':
        labels_list_nao[i] = 'Fulltime'


pose_keypoints_list_5 = np.load('/content/drive/MyDrive/ModelFiles/padding/pose_keypoints_list_thunder_files_pad.npy')
labels_list_5 = np.load('/content/drive/MyDrive/ModelFiles/padding/labels_list_thunder_files_pad.npy')


pose_keypoints_list_6 = np.load('/content/drive/MyDrive/ModelFiles/padding/pose_keypoints_list_thunder_Newflip.npy')
labels_list_6 = np.load('/content/drive/MyDrive/ModelFiles/padding/labels_list_thunder_Newflip.npy')
#pose_keypoints_list_4 = np.load('/content/drive/MyDrive/ModelFiles/augmented_skeleton_keypoints_500.npy', allow_pickle=True)
#labels_list_4 = np.load('/content/drive/MyDrive/ModelFiles/augmented_labels_500.npy', allow_pickle=True)

#pose_keypoints_list_7 = np.load('/content/drive/MyDrive/ModelFiles/augmented_skeleton_keypoints_500_02tr.npy', allow_pickle=True)
#labels_list_7 = np.load('/content/drive/MyDrive/ModelFiles/augmented_labels_500_02tr.npy', allow_pickle=True)


pose_keypoints_list_8 = np.load('/content/drive/MyDrive/ModelFiles/padding/pose_keypoints_list_thunder_nao2_pad.npy')
labels_list_8 = np.load('/content/drive/MyDrive/ModelFiles/padding/labels_list_thunder_nao2_pad.npy')

for i, item in enumerate(labels_list_8):
    if item == 'corner-kick-red':
        labels_list_8[i] = 'Corner-kick Red'
    elif item == 'corner-kick-blue':
        labels_list_8[i] = 'Corner-kick Blue'
    elif item == 'goal-red':
        labels_list_8[i] = 'Goal Red'
    elif item == 'goal-blue':
        labels_list_8[i] = 'Goal Blue'
    elif item == 'goal-kick-red':
        labels_list_8[i] = 'Goal-kick Red'
    elif item == 'goal-kick-blue':
        labels_list_8[i] = 'Goal-kick Blue'
    elif item == 'kick-in-red':
        labels_list_8[i] = 'Kick-in Red'
    elif item == 'kick-in-blue':
        labels_list_8[i] = 'Kick-in Blue'
    elif item == 'player-exchange-red':
        labels_list_8[i] = 'Player exchange Red'
    elif item == 'player-exchange-blue':
        labels_list_8[i] = 'Player exchange Blue'
    elif item == 'pushing-free-kick-red':
        labels_list_8[i] = 'Pushing Free-kick Red'
    elif item == 'pushing-free-kick-blue':
        labels_list_8[i] = 'Pushing Free-kick Blue'
    elif item == 'full-time':
        labels_list_8[i] = 'Fulltime'


dataset_vars = [
    [pose_keypoints_list_5,
     labels_list_5],
    [pose_keypoints_list_6,
     labels_list_6]

]


split_amount = 0.8
X_train, X_val, y_train, y_val = preprocess_datasets(dataset_vars, split_amount)

class_counts = tf.math.reduce_sum(y_train, axis=0)
for i, count in enumerate(class_counts):
    print(f"Class {i}: {count}")


class_counts = tf.math.reduce_sum(y_val, axis=0)

# Print the number of datapoints per class
for i, count in enumerate(class_counts):
    print(f"Class {i}: {count}")



print(X_train)
print(y_train)

In [ ]:
#training the model
from tensorflow.keras.callbacks import LearningRateScheduler

# Define LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(24,18), return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(Flatten())
model.add(Dense(units=len(label_mapping), activation='softmax'))

# Define the initial learning rate and the function to update it
initial_learning_rate = 0.5

def learning_rate_scheduler(epoch, learning_rate):
    factor = 1
    if epoch == 100 * factor:  # Adjust this epoch threshold as per your needs
        factor += 1
        return learning_rate * 0.8  # Reduce the learning rate
    return learning_rate

# Compile the model with initial learning rate
optimizer = tf.keras.optimizers.Adam(lr=initial_learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(learning_rate_scheduler)

# Fit the model with the learning rate scheduler callback
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=1024, callbacks=[lr_scheduler])
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation loss:", loss)
print("Validation accuracy:", accuracy)

# Plot the training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# eval on the test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Calculate accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)

# Plot the confusion matrix with accuracy
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_mapping, yticklabels=label_mapping)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix\nAccuracy: {accuracy:.2f}')
plt.show()